<a href= "https://hrs.isr.umich.edu/sites/default/files/meta/2002/core/codebook/h02_00.html?_ga=2.14020593.714056361.1676427746-1610833755.1676427746">codebook</a>

In [74]:
import glob
import pandas as pd
from functools import reduce

all_dispositon_files = glob.glob('hrs_data/disposition/*.csv')
lst_disposition = []
for filename in all_dispositon_files:
    df_disposition = pd.read_csv(filename, index_col=None, header=0)
    lst_disposition.append(df_disposition)
dispositon_data = pd.concat(lst_disposition, axis=0,
                            ignore_index=True).iloc[:, :2]
dispositon_data['deceased'] = 1

In [79]:
all_core_files = glob.glob('hrs_data/*.csv')
lst_core = []
for filename in all_core_files:
    dfcore = pd.read_csv(filename, index_col=None, header=0)
    lst_core.append(dfcore)
core_data = reduce(lambda  left,right: pd.merge(left,right,on=['HHID'],
                                            how='outer'), lst_core)
core_data

,HHID,HSUBHH_x,GSUBHH_x,HPN_CS,HPN_FAM,HPN_FIN,HPN_NCS,HPN_NFAM,HPN_NFIN,HQ010,...,HC176,HC177,HC178,HC179,HC180,HC181,HC182,HC183,HVERSION_y,HQNR_y
0,3,0,0,20,20,10.0,10.0,10.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,300010
1,3,0,0,20,20,10.0,10.0,10.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,300020
2,10001,0,0,10,10,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1000100010
3,10003,0,0,30,30,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1000300030
4,10004,0,0,10,40,10.0,40.0,10.0,40.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1000400010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18813,213468,0,0,10,10,10.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,21346800010
18814,213471,0,0,10,10,10.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,21347100010
18815,213475,0,0,10,10,10.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,21347500010
18816,213479,0,0,20,20,10.0,10.0,10.0,20.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,21347900010


In [80]:
df02_d = core_data.merge(dispositon_data, on=["HHID", "PN"], how="left")
df02_d['deceased'] = df02_d['deceased'].fillna(0)
df02_d['deceased'].value_counts()

0.0    18081
1.0      814
Name: deceased, dtype: int64